# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Program: _Computer Systems Engineering_  </center>
---
### <center> **Spring 2025** </center>
---

**Lab 06**: Big Data Pipeline for Netflix data.

**Date**: 10/03/2025

**Team Name**: Par De Dos

**Professor**: Pablo Camarillo Ramirez

In [2]:
# Initialize findspark to get acces to de PySpark installation
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

# Create connection to the spark cluster
spark = SparkSession.builder \
    .appName("Lab 05 - Par de Dos") \
    .master("spark://7a359a1e9836:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()

# Create SparkContext
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/11 01:37:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from team_ParDeDos.spark_utils import SparkUtils
netflix_schema = SparkUtils.generate_schema([
	("show_id", "string"),
	("type", "string"),
	("title", "string"),
	("director", "string"),
	("country", "string"),
	("rating", "string"),
	("duration", "string"),
	("listed_in", "string"),
	("release_year", "integer"),
	("date_added", "date")
])

netflix_df = spark.read.schema(netflix_schema).option("header", "true").csv("/home/jovyan/notebooks/data/netflix1.csv")

In [6]:
netflix_df.printSchema()
netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- date_added: date (nullable = true)



+-------+-------+--------------------------------+---------------+-------------+---------+--------+---------+------------+----------+
|show_id|type   |title                           |director       |country      |rating   |duration|listed_in|release_year|date_added|
+-------+-------+--------------------------------+---------------+-------------+---------+--------+---------+------------+----------+
|s1     |Movie  |Dick Johnson Is Dead            |Kirsten Johnson|United States|9/25/2021|2020    |PG-13    |NULL        |NULL      |
|s3     |TV Show|Ganglands                       |Julien Leclercq|France       |9/24/2021|2021    |TV-MA    |NULL        |NULL      |
|s6     |TV Show|Midnight Mass                   |Mike Flanagan  |United States|9/24/2021|2021    |TV-MA    |NULL        |NULL      |
|s14    |Movie  |Confessions of an Invisible Girl|Bruno Garotti  |Brazil       |9/22/2021|2021    |TV-PG    |NULL        |NULL      |
|s8     |Movie  |Sankofa                         |Haile Gerima

In [8]:
netflix_cleaned = SparkUtils.clean_df(netflix_df, netflix_schema)

netflix_cleaned.printSchema()
netflix_cleaned.show(5, truncate=False)


TypeError: cannot unpack non-iterable StructField object

In [1]:
# Stop the SparkContext
sc.stop()

NameError: name 'sc' is not defined